# Transformers Basics

Date: 2023/11/14 - 2023/11/16

参考書: https://gihyo.jp/book/2023/978-4-297-13633-8

この本の１章に書かれるちょっとしたコードを実行するだけで、NLPの全体像をなんとなく理解できた。
私にとっては、文章分類、意味的類似度計算、固有表現認識、要約生成あたりが有用に思える。
この程度のNLPでは、Google Colabに頼らずとも、MacBook Air上でサッと動くのが良いです。

In [1]:
#!pip3 install 'transformers[ja,torch]' datasets matplotlib japanize-matplotlib
#!pip3 install Xformers
import warnings
warnings.filterwarnings('ignore')

In [2]:
from transformers import pipeline

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


## 1.1.1 文章分類

In [3]:
text_classification_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-marc_ja')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
positive_text = '世界には言葉がわからなくても感動する音楽がある。'
negative_text = '世界には言葉がでないほどひどい音楽たある。'
print(text_classification_pipeline(positive_text))
print(text_classification_pipeline(negative_text))

[{'label': 'positive', 'score': 0.9993619322776794}]
[{'label': 'negative', 'score': 0.988598108291626}]


In [5]:
import yaml
with open('data/impressions.yaml', 'r') as f:
    imp = yaml.safe_load(f)

sents = [['positive_negative', s] for s in imp['positive_negative']]
sents.extend([['positive', s] for s in imp['positive']])
sents.extend([['negative', s] for s in imp['negative']])
sents

[['positive_negative', 'ショールームの清潔さには好感が持てましたが、一部の展示品が傷んでいるのが気になりました。'],
 ['positive_negative', 'スタッフの丁寧な対応がありましたが、商品の説明が少なく、理解しにくかったです。'],
 ['positive_negative', '展示品の品質は高く、特にデザインに魅了されましたが、価格が高すぎると感じました。'],
 ['positive_negative', 'ショールームの広さとレイアウトは素晴らしかったが、一部のエリアが混雑していました。'],
 ['positive_negative', '商品のバリエーションが豊富で、選択肢が多い一方で、一部のカテゴリーが不足していました。'],
 ['positive_negative', 'インテリアデザインがモダンで印象的でしたが、一部のエリアが古く感じられました。'],
 ['positive_negative', 'スタッフの対応は親切で好感が持てましたが、混雑しておりゆっくり見学できませんでした。'],
 ['positive_negative', '展示品のディスプレイは美しく、高い品質が感じられましたが、価格が高すぎると感じました。'],
 ['positive_negative', 'ショールーム内の照明が良好で、商品が見やすかったが、一部のエリアが薄暗かったです。'],
 ['positive_negative', '商品の価格設定が適切で良心的でしたが、スタッフの知識が不足しているように感じました。'],
 ['positive_negative', '展示品の配置が工夫され、テーマ性がありましたが、一部の商品が見つけにくかったです。'],
 ['positive_negative', 'ショールーム内の案内が分かりやすく、スタッフの対応も良かったが、混雑が気になりました。'],
 ['positive_negative', '商品の新しさとデザインに魅了されましたが、スタッフの態度が冷淡で残念でした。'],
 ['positive_negative', 'ショールームの雰囲気が明るく、居心地が良かったが、一部の商品が陳腐に感じられました。'],
 ['positive_negative', '展示品の保守

In [6]:
import time

classified = []

start = time.time()
for sent in sents:
    cl = text_classification_pipeline(sent[1])[0]
    l = [cl['label'], cl['score'], sent[0], sent[1]]
    classified.append(l)
    print(l)
finish = time.time()

print('\nThe number of sentences: {}'.format(len(sents)))
print('Elapsed time: {} sec'.format(finish - start))
print('For each word: {:.01f} msec'.format((finish - start)/len(sents)*1000))

['negative', 0.9472939372062683, 'positive_negative', 'ショールームの清潔さには好感が持てましたが、一部の展示品が傷んでいるのが気になりました。']
['negative', 0.9519743919372559, 'positive_negative', 'スタッフの丁寧な対応がありましたが、商品の説明が少なく、理解しにくかったです。']
['negative', 0.8354621529579163, 'positive_negative', '展示品の品質は高く、特にデザインに魅了されましたが、価格が高すぎると感じました。']
['positive', 0.9290488958358765, 'positive_negative', 'ショールームの広さとレイアウトは素晴らしかったが、一部のエリアが混雑していました。']
['positive', 0.9929064512252808, 'positive_negative', '商品のバリエーションが豊富で、選択肢が多い一方で、一部のカテゴリーが不足していました。']
['positive', 0.9283452033996582, 'positive_negative', 'インテリアデザインがモダンで印象的でしたが、一部のエリアが古く感じられました。']
['negative', 0.9903730154037476, 'positive_negative', 'スタッフの対応は親切で好感が持てましたが、混雑しておりゆっくり見学できませんでした。']
['negative', 0.9414645433425903, 'positive_negative', '展示品のディスプレイは美しく、高い品質が感じられましたが、価格が高すぎると感じました。']
['positive', 0.9860449433326721, 'positive_negative', 'ショールーム内の照明が良好で、商品が見やすかったが、一部のエリアが薄暗かったです。']
['negative', 0.9144169688224792, 'positive_negative', '商品の価格設定が適切で良心的でしたが、スタッフの知識が不足しているように感じました。']
['posi

## 1.1.2 自然言語推論

In [7]:
nli_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-jnli')
text = '二人の男性がジェット機を見ています'
entailment_text = 'ジェット機を見ている人が二人います'
print(nli_pipeline({'text': text, 'text_pair': entailment_text}))

{'label': 'entailment', 'score': 0.9964311122894287}


In [8]:
contradiction_text='二人の男性が飛んでいます'
print(nli_pipeline({'text': text, 'text_pair': contradiction_text}))

{'label': 'contradiction', 'score': 0.9990535378456116}


In [9]:
neutral_text = '２人の男性が、白い飛行機を眺めています'
print(nli_pipeline({'text': text, 'text_pair': neutral_text}))

{'label': 'neutral', 'score': 0.9959145188331604}


## 1.1.3 意味的類似度計算

In [10]:
text_sim_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-jsts', function_to_apply='none')

In [11]:
text = '川べりでサーフボードを持った人たちがいます'
sim_text = 'サーファー達が川べりに立っています'
result = text_sim_pipeline({'text': text, 'text_pair': sim_text})
print(result['score'])

3.60836124420166


In [12]:
dissem_text = 'トイレの壁に黒いタオルがかけられています'
result = text_sim_pipeline({'text': text, 'text_pair': dissem_text})
print(result['score'])

0.04162156581878662


In [13]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-unsup-simcse-jawiki',
    task='feature-extraction'
)

text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8540017008781433


## 1.1.4 固有表現認識

In [14]:
ner_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-ner-wikipedia-dataset',
    aggregation_strategy='simple'
)
text='大谷翔平選手は岩手県水沢市出身のプロ野球選手'
ner_pipeline(text)

[{'entity_group': '人名',
  'score': 0.99810076,
  'word': '大谷 翔平',
  'start': None,
  'end': None},
 {'entity_group': '地名',
  'score': 0.99878407,
  'word': '岩手 県 水沢 市',
  'start': None,
  'end': None}]

In [15]:
# [引用] https://ja.wikipedia.org/wiki/%E3%83%93%E3%83%BC%E3%83%88%E3%83%AB%E3%82%BA
text='The Beatlesは、1960年代から1970年にかけて活動したイギリス・リヴァプール出身のロックバンド、および20世紀を代表する音楽グループである。音楽誌『ローリング・ストーン』による「ローリング・ストーンの選ぶ歴史上最も偉大な100組のアーティスト」において第1位にランクインしており[7]、経済紙ウォール・ストリート・ジャーナルの統計算出に基づく「史上最も人気のある100のロックバンド」でも1位となっている[8]。グラミー賞を7回受賞し、23回ノミネートされている[9]。'
[item['word'] for item in ner_pipeline(text) if item['score'] >= 0.9]

['The Beatles', 'イギリス', 'リヴァプール', 'ローリング ・ ストーン', 'グラミー 賞']

## 1.1.5 要約生成

In [16]:
text2text_pipeline=pipeline(
    model='llm-book/t5-base-long-livedoor-news-corpus'
)

import re

# [引用]  https://ja.wikipedia.org/wiki/%E3%83%93%E3%83%BC%E3%83%88%E3%83%AB%E3%82%BA
article='「BEATLES」という名称はジョン・レノンとスチュアート・サトクリフが考えた造語である。レノンの証言では[13]、この名前が考案されたのは1960年の4月で、バディ・ホリーのバンド名「バディ・ホリー&ザ・クリケッツ」のクリケッツ[注釈 6] にあやかり、昆虫の名前で同じように2つの意味を含んでいる言葉としてビートルズ[注釈 7] を、映画『乱暴者』[注釈 8] から思いついた。しかし、バンド名を「ビートルズ」とした頃、クラブ出演を依頼してきたブライアン・キャス[13]は難色を示し[注釈 9]、改名を出演条件として「ロング・ジョン&ピーシズ・オブ・シルヴァー」という名称を提示した。話し合いの上、互いに譲歩して「ロング・ジョン&シルヴァー・ビートルズ」と称することになったが、その後ロング・ジョンを除いて「シルヴァー・ビートルズ」と称した[注釈 10][13]。ただし、1960年8月から行った最初のハンブルク巡業で出演したクラブ「カイザー・ケラー」の広告[14]には「The Beatles」と記載されている。'
article = re.sub(r'\[\d+\]|\[注釈\s\d+\]', '', article)
text2text_pipeline(article)

[{'generated_text': '「BEATLES」はビートルズの略称'}]

要約のほうはイマイチ。。。